In [1]:
import numpy as np

# Poisson Regression

This is an example of an application of Convex Optimization found in the [Convex Optimization textbook](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf) by Boyd and Vandenberghe. There is a great [online course](https://www.edx.org/course/convex-optimization) offered on EdX which I highly recommend. 

Suppose $Y$ is a Poisson random variable with mean $\mu$:

$$ P(Y=k) = \frac{\mu^k e^{-k}}{k!} $$

Suppose the mean $\mu$ is a linear combination of some explanatory variables:

$$ \mu = a^Tx + b$$

and that we have some data $(x_i,y_i)$ for $i=1,2,...,m$ where $x_i \in \mathbb{R}^n$ is a vector of explanatory variable values and $y_i$ is the response variable. We want to find MLE's of the parameters $a$ and $b$.


The log likelihood is

$$ \displaystyle\sum_{i=1}^m\left( y_i\log(a^Tx_i+b) - (a^Tx_i+b) - \log(y_i!) \right)$$

Instead of maximizing the above log likelihood, we will solve the problem of minimizing 

$$ \displaystyle\sum_{i=1}^m a^Tx_i+b - y_i\log(a^Tx_i+b)$$

We will compile all of the observed data into a single design matrix, where we also include a column of ones for the intercept $b$. We wish to minimize

$$ \displaystyle\sum_{i=1}^m \left[(Xz)_i - y_i \log((Xz)_i) \right] $$

where $X$ is the matrix whose rows are $x_1, x_2, ..., x_m$, with an additional column of ones for the intercept.

**Note:** In many other textbooks, Poisson regression is stated as follows. 

$$E\left[Y|x\right] = \mu \qquad \text{ and } \qquad \log(\mu) = Xz$$

where $z$ is again the parameter vector we wish to estimate. So the assumption is the log of the expected value is a linear function of the parameter vector $z$. 

In [2]:
# generate test data
n = 4 # number of explanatory variables
m = 20000 # number of data points
real_z = np.append(10*np.random.uniform(-1,1,n), 50) # last entry of z represents the intercept
# design matrix
X = np.concatenate([np.random.beta(2,5,(m,1)), np.random.normal(1,0.25, (m,1)), 
                    np.random.binomial(1,0.7,(m,1)), np.random.uniform(1,2,(m,1)),
                   np.ones((m,1))], axis=1)
real_mu = X@real_z
y = np.random.poisson(real_mu, m)

In [3]:
real_z

array([-1.26866953,  2.37727535,  7.84591618, -6.20268071, 50.        ])

In [4]:
def nll(X,y,z):
    return np.sum((X@z)-y*np.log(X@z))
def grad(X,y,z):
    return X.T@(1 -y/(X@z))
def backtrack(obj,X,y,z, gradient, alpha, beta):
    t = 1
    while np.min(X@(z-t*gradient)) <= 0 :
        t = t*beta
    while obj(X,y,z - t*gradient) >  obj(X,y,z) - alpha*t*gradient@gradient:
        t = t*beta
    return t
def l2_norm(x):
    return np.sqrt(np.sum(x**2))

In [5]:
# Gradient Descent
z = np.append(np.zeros(n),0.1) # initial point
tol = 1e-8 # stopping criterion tolerance for L2 norm of gradient
max_iter = 1000
# parameters 
alpha = 0.1 # line search
beta = 0.5 # line search
i = 0
while i <= max_iter:
    g = grad(X,y,z)
    if l2_norm(g) < tol:
        break
    else:
        t = backtrack(nll, X, y, z, g, alpha, beta) # step length
        z = z - t*g # update
        i += 1

In [6]:
z

array([-1.18370646,  2.30877419,  7.98245933, -6.06223243, 49.70311673])

In [7]:
def hessian(X,y,z):
    return X.T@np.diag(y/((X@z)**2))@X

In [8]:
def NM_backtrack(obj,X,y,z, gradient, newton_step, alpha, beta):
    t = 1
    while np.min(X@(z+t*newton_step)) <= 0 :
        t = t*beta
    while obj(X,y,z+t*newton_step) >  obj(X,y,z) + alpha*t*gradient@newton_step:
        t = t*beta
    return t

In [9]:
# Newton's Method
z = np.append(np.zeros(n),0.1)
tol = 1e-8 # stopping criterion tolerance for L2 norm of gradient
max_iter = 100
# parameters 
alpha = 0.1 # line search
beta = 0.8 # line search
i = 0
while i <= max_iter:
    g = grad(X,y,z)
    H = hessian(X,y,z)
    newton_step = -np.linalg.inv(H)@g
    dec = -g@newton_step
    if dec < tol:
        break
    else:
        t = NM_backtrack(nll, X, y,z, g, newton_step, alpha, beta)
        z = z+t*newton_step
    i +=1

In [10]:
z

array([-1.21260883,  2.28618061,  7.97928613, -6.08617612, 49.77380631])

In [11]:
# confidence interval for parameter vector z
covar = np.linalg.inv(X.T@np.diag(X@z)@X)
lower_end = z -  2*np.exp(np.sqrt(np.diag(covar)))
upper_end = z +  2*np.exp(np.sqrt(np.diag(covar)))
for i in range(len(lower_end)):
    print('A 95% confidence interval for the', i,'th entry of the paramter vector is', round(lower_end[i],4), 'to', round(upper_end[i],4))

A 95% confidence interval for the 0 th entry of the paramter vector is -3.2253 to 0.8001
A 95% confidence interval for the 1 th entry of the paramter vector is 0.278 to 4.2944
A 95% confidence interval for the 2 th entry of the paramter vector is 5.9747 to 9.9839
A 95% confidence interval for the 3 th entry of the paramter vector is -8.0933 to -4.0791
A 95% confidence interval for the 4 th entry of the paramter vector is 47.7593 to 51.7883
